In [1]:
from api_calls import *
import numpy as np

In [2]:
mongo_db = 'withTheSpread'  # Replace with your desired database name
mongo_collection = 'season_spreads'  # Replace with your desired collection name
from api_calls import get_db, mongoConn

client = mongoConn()
dfseasonspreads = get_db(client, mongo_db, mongo_collection)
dfseasonspreads = dfseasonspreads.set_index(['Team'], drop=True).drop(['_id'], axis=1)

Pinged your deployment. You successfully connected to MongoDB!


In [8]:
dfplot = dfseasonspreads.copy()
def checkcover(x,metric):
    if 'pred' in metric:
        result = np.nan if (pd.isna(x[[metric,'spreadscore']]).any()) else (np.sign(x['spreadscore'])==np.sign(round(x[metric],2))).astype(int)
    elif 'cover' in metric:
        result = np.nan if (pd.isna(x[[metric,'spreadscore']]).any()) else (np.sign(x['spreadscore'])==np.sign(x[metric]-.5)).astype(int)
    else:
        return 'Not valid metric'
    return result
prediction_cols = ['predspread','predspread_diff','coverprob','coverprob_diff']
for col in prediction_cols:
    dfplot[col+'win'] = dfplot.apply(lambda row: checkcover(row,col),axis=1)
dfplot.groupby(['Year','Week'])[['predspreadwin','predspread_diffwin','coverprobwin','coverprob_diffwin']].mean()

predspreadwin  predspread_diffwin  coverprobwin  coverprob_diffwin
Year Week                                                                    
2023 1               NaN                 NaN           NaN                NaN
     2          0.500000                 NaN      0.468750           0.468750
     3          0.531250            0.312500      0.468750           0.500000
     4          0.375000            0.437500      0.500000           0.500000
     5          0.428571            0.357143      0.392857           0.464286
     6          0.433333            0.533333      0.433333           0.500000
     7          0.307692            0.230769      0.615385           0.500000
     8          0.343750            0.437500      0.500000           0.437500
     9          0.392857            0.214286      0.392857           0.464286
     10         0.500000            0.541667      0.416667           0.541667
     11         0.416667            0.500000      0.500000           0.500000
     12         0.468750            0.437500      0.500000           0.500000
     13         0.541667            0.500000      0.500000           0.458333
     14         0.500000            0.461538      0.461538           0.461538
     15         0.468750            0.250000      0.437500           0.437500
     16         0.538462            0.653846      0.576923           0.423077
     17         0.437500            0.312500      0.406250           0.500000
     18         0.562500            0.500000      0.593750           0.500000
2024 1          0.437500            0.562500      0.562500           0.500000
     2          0.375000            0.437500      0.375000           0.500000
     3          0.531250            0.625000      0.500000           0.500000
     4          0.406250            0.312500      0.531250           0.468750
     5          0.500000            0.571429      0.464286           0.500000
     6          0.571429            0.428571      0.607143           0.464286
     7          0.533333            0.666667      0.633333           0.533333
     8               NaN                 NaN           NaN                NaN

In [7]:
dfplot.groupby('Year')[['predspreadwin','predspread_diffwin','coverprobwin','coverprob_diffwin']].mean()

,predspreadwin,predspread_diffwin,coverprobwin,coverprob_diffwin
Year,,,,
2023,0.455285,0.413043,0.479675,0.479675
2024,0.476636,0.514019,0.523364,0.495327


In [12]:
dfplot['cover'] = np.sign(dfplot['spreadscore'])
dfplot[['spreadscore','cover']+prediction_cols].corr()[['spreadscore','cover']]

,spreadscore,cover
spreadscore,1.000000,0.774087
cover,0.774087,1.000000
predspread,-0.084732,-0.076593
predspread_diff,-0.128364,-0.108838
coverprob,0.024158,0.005941
coverprob_diff,0.036788,0.022597


In [59]:
dfseasonspreads

,Week,opponent,spread,score,diff,Year,spreadscore,coverprob,predspread,order,coverprob_diff,predspread_diff
Team,,,,,,,,,,,,
Washington Commanders,1,Arizona Cardinals,-7.0,20.0,4.0,2023,-3.0,NaN,NaN,NaN,NaN,NaN
Chicago Bears,1,Green Bay Packers,-1.5,20.0,-18.0,2023,-19.5,NaN,NaN,NaN,NaN,NaN
Arizona Cardinals,1,Washington Commanders,7.0,16.0,-4.0,2023,3.0,NaN,NaN,NaN,NaN,NaN
Denver Broncos,1,Las Vegas Raiders,-3.0,16.0,-1.0,2023,-4.0,NaN,NaN,NaN,NaN,NaN
Tennessee Titans,1,New Orleans Saints,3.0,15.0,-1.0,2023,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
San Francisco 49ers,10,Jacksonville Jaguars,-3.0,NaN,NaN,2023,NaN,0.603914,0.008170,5.0,0.154278,-2.775560
Seattle Seahawks,10,Washington Commanders,-6.0,NaN,NaN,2023,NaN,0.348096,-0.361575,11.0,-0.219533,-0.089633
Tampa Bay Buccaneers,10,Tennessee Titans,-1.5,NaN,NaN,2023,NaN,0.500704,-0.115931,7.0,-0.218272,-0.635019
